In [11]:
import pickle
import pandas as pd
import numpy as np
from IPython.display import Audio
import torch
import torchaudio
from IPython.display import Audio, display
import json
import os
from pathlib import Path

In [12]:
def get_df(dir):
    exp_dir = Path("/data/ephraim/datasets/known_noise/undiff/exp_ar_i_real")/dir
    pkl_results_file = os.path.join(exp_dir, "measures.pickle")
    # print(pkl_results_file)

    with open(pkl_results_file, "rb") as handle:
        dfme = pd.read_pickle(handle)
    return dfme

In [13]:

pkl_results_file = "/data/ephraim/datasets/known_noise/sgmse/exp_i/storm_vbd/enhanced_real/measures_storm.pickle"

with open(pkl_results_file, "rb") as handle:
    df2 = pd.read_pickle(handle)
for i in range(len(df2["snr"])):
    df2.at[i, "snr"] = df2["snr"][i].split("_")[0]

In [14]:
df_storm_5 = df2[df2["snr"]=="5"].reset_index(drop=True).sort_values(by=['dir'])

In [15]:
df_storm_5["noise_type"][2]

'Factory'

In [16]:
pkl_results_file = "/data/ephraim/datasets/known_noise/sgmse/exp_i/sgmse_WSJ0Chime3/enhanced_real/measures_storm.pickle"

with open(pkl_results_file, "rb") as handle:
    df1 = pd.read_pickle(handle)
for i in range(len(df1["snr"])):
    df1.at[i, "snr"] = df1["snr"][i].split("_")[0]

In [17]:
df_sg_5 = df1[df1["snr"]=="5"].reset_index(drop=True).sort_values(by=['dir'])

In [18]:
df_sg_5["noise_type"][2]

'Factory'

In [19]:
dfme[dfme["snr"]=="5"][["stoi","pesq","name","sdr","noise_type"]].head()

,stoi,pesq,name,sdr,noise_type
0,0.415153,1.066994,s0.08,2.235773,Car
1,0.455589,1.135010,s0.08,7.044814,Babble
2,0.443848,1.093369,s0.08,6.416396,Room
3,0.451190,1.092926,s0.08,6.945102,Factory
4,0.458007,1.103583,s0.08,6.929780,Speech


In [20]:
dfme = get_df("b")
dfme[dfme["snr"]=="5"][["stoi","pesq","name","sdr","noise_type"]].head()

,stoi,pesq,name,sdr,noise_type
0,0.913403,1.147228,s0.08,3.788308,Car
1,0.917692,1.257722,s0.08,10.377759,Babble
2,0.891717,1.284066,s0.08,9.172256,Room
3,0.882190,1.287000,s0.08,10.487592,Factory
4,0.894608,1.244149,s0.08,10.120212,Speech


In [21]:
mine = None
df_noisy = None
for d in "abcdefghij": 
    if get_df(d) is None:
        continue
    dfme = get_df(d)
    dfme = dfme.assign(dir=d)
    cur_df_noisy = dfme[dfme["name"]=="noisy"].reset_index(drop=True)
    cur_df_noisy.loc[0,"dir"] = d
    if df_noisy is None:
        df_noisy = cur_df_noisy
    else:
        df_noisy = pd.concat([df_noisy, cur_df_noisy])
    dfme = dfme[dfme["name"] != "noisy"]
    dfme_5 = dfme[dfme["snr"]=="5"]
    dfme5 = dfme_5[dfme_5["stoi"]>0.5]
    cols =["dir","name","stoi","input_stoi","pesq","input_pesq","OVRL","SIG","BAK","si_sdr",'sdr', "sar",'sir',"noise_type", "filename"]
    NOISES = ["Car","Babble","Room","Factory","Speech"]
    for noisetype in NOISES :
        cur_mine = dfme5[dfme5["noise_type"]==noisetype]
        cur_mine = cur_mine[cur_mine["pesq"] ==cur_mine["pesq"].max()].reset_index(drop=True)[cols]
        # cur_mine.loc[0,"dir"] = d
        if mine is None:
            mine = cur_mine
        else:
            mine = pd.concat([mine, cur_mine])
mine=mine.reset_index(drop=True)
df_noisy
# dfme5[dfme5["pasq"]][["stoi","pesq","name","si_sdr","OVRL","SIG","BAK"]]

,input_si_sdr,input_sdr,input_sir,input_sar,input_stoi,input_pesq,si_sdr,sdr,sir,sar,...,sr,num_hops,OVRL_raw,SIG_raw,BAK_raw,OVRL,SIG,BAK,P808_MOS,dir
0,3.551430,3.592592,inf,3.592592,0.924115,1.305760,3.551430,3.592592,inf,3.592592,...,16000,8,2.645686,4.073503,2.269344,2.522964,3.584308,2.577027,3.610453,a
1,3.519935,3.555063,inf,3.555063,0.720964,1.057805,3.519935,3.555063,inf,3.555063,...,16000,8,1.004084,1.049723,1.032766,1.097817,1.194144,1.125380,2.705957,a
2,3.462998,3.490358,inf,3.490358,0.758443,1.053132,3.462998,3.490358,inf,3.490358,...,16000,8,0.984259,1.019282,1.040191,1.078380,1.162377,1.135307,2.276417,a
3,3.492102,3.524646,inf,3.524646,0.699227,1.032454,3.492102,3.524646,inf,3.524646,...,16000,8,1.358124,2.530941,1.206424,1.435201,2.542313,1.352995,2.202747,a
4,3.496120,3.534893,inf,3.534893,0.719912,1.038966,3.496120,3.534893,inf,3.534893,...,16000,8,1.046008,1.323078,1.035789,1.138464,1.465914,1.129323,2.252606,a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,8.468923,8.472498,inf,8.472498,0.980609,2.135419,8.468923,8.472498,inf,8.472498,...,16000,1,3.319140,4.115535,3.342615,3.002989,3.607352,3.511595,3.912681,j
6,8.503744,8.510720,inf,8.510720,0.871973,1.366439,8.503744,8.510720,inf,8.510720,...,16000,1,2.176779,3.180040,2.048342,2.153534,3.038375,2.347611,3.193241,j
7,8.505436,8.523503,inf,8.523503,0.884725,1.354712,8.505436,8.523503,inf,8.523503,...,16000,1,1.639577,2.473068,1.495572,1.693023,2.510880,1.716053,2.931324,j
8,8.501519,8.507142,inf,8.507142,0.871656,1.235686,8.501519,8.507142,inf,8.507142,...,16000,1,2.424149,3.676100,2.272950,2.352457,3.358388,2.581243,2.755824,j


In [22]:
for noisetype in NOISES:
    noise_mine  = mine[mine["noise_type"]==noisetype]
    noise_mine.to_excel(f"/data/ephraim/datasets/known_noise/sgmse/exp_i/analysis/i_real/ours_{noisetype}.xlsx")


In [27]:
for noisetype in NOISES:
    noise_df_sg_5 = df_sg_5[df_sg_5["noise_type"]==noisetype]
    noise_df_sg_5[cols].to_excel(f"/data/ephraim/datasets/known_noise/sgmse/exp_i/analysis/i_real/sgmse_{noisetype}.xlsx")
# df_sg_5[cols].describe()[1:3].to_excel("/data/ephraim/datasets/known_noise/sgmse/exp_i/analysis/sgmse_stats.xlsx")

In [28]:
for noisetype in NOISES:
    noise_df_storm_5 = df_storm_5[df_storm_5["noise_type"]==noisetype]
    noise_df_storm_5[cols].to_excel(f"/data/ephraim/datasets/known_noise/sgmse/exp_i/analysis/i_real/storm_{noisetype}.xlsx")
    # df_storm_5[cols].describe()[1:3].to_excel("/data/ephraim/datasets/known_noise/sgmse/exp_i/analysis/storm_stats.xlsx")

In [23]:
mine.head()

,dir,name,stoi,input_stoi,pesq,input_pesq,OVRL,SIG,BAK,si_sdr,sdr,sar,sir,noise_type,filename
0,a,s0.3,0.939825,0.924115,1.383912,1.305760,2.659904,3.536586,2.907966,4.052824,4.122058,4.122058,inf,Car,/data/ephraim/datasets/known_noise/undiff/exp_...
1,a,s0.11,0.823307,0.720964,1.280107,1.057805,3.182914,3.478178,4.028701,9.817067,10.241671,10.241671,inf,Babble,/data/ephraim/datasets/known_noise/undiff/exp_...
2,a,s0.12,0.817155,0.758443,1.240639,1.053132,2.381563,3.518393,2.331776,8.819594,9.239682,9.239682,inf,Room,/data/ephraim/datasets/known_noise/undiff/exp_...
3,a,s0.11,0.799380,0.699227,1.310608,1.032454,2.753213,3.355914,3.358786,9.963297,10.560804,10.560804,inf,Factory,/data/ephraim/datasets/known_noise/undiff/exp_...
4,a,s0.1,0.817351,0.719912,1.360432,1.038966,3.067470,3.371378,3.958709,10.687693,11.307800,11.307800,inf,Speech,/data/ephraim/datasets/known_noise/undiff/exp_...


In [24]:
for i in range(len(df_storm_5["dir"])):
    dir_ = df_storm_5["dir"][i]
    noisetype = df_storm_5["noise_type"][i]
    mine_parallel = mine[(mine["dir"]==dir_) & (mine["noise_type"]==noisetype)]
    if len(mine_parallel)==0:
        idx = df_storm_5[(df_storm_5.dir==dir_)&(df_storm_5.noise_type==noisetype)].index
        df_storm_5 = df_storm_5.drop(idx)

In [25]:
for i in range(len(df_sg_5["dir"])):
    dir_ = df_sg_5["dir"][i]
    noisetype = df_sg_5["noise_type"][i]
    mine_parallel = mine[(mine["dir"]==dir_) & (mine["noise_type"]==noisetype)]
    if len(mine_parallel)==0:
        idx = df_sg_5[(df_sg_5.dir==dir_)&(df_sg_5.noise_type==noisetype)].index
        df_sg_5 = df_sg_5.drop(idx)

In [26]:
minestats = mine.describe()[1:3]
minestats = minestats.assign(alg="ours")
df_storm_5stats = df_storm_5[cols].describe()[1:3]
df_storm_5stats = df_storm_5stats.assign(alg="storm")
df_sg_5stats = df_sg_5[cols].describe()[1:3]
df_sg_5stats = df_sg_5stats.assign(alg="sgmse")
statsdf = pd.concat([minestats,df_sg_5stats,df_storm_5stats])
statsdf.to_excel("/data/ephraim/datasets/known_noise/sgmse/exp_i/analysis/i_real/all_stats.xlsx")
# 

/data/ephraim/downloads/anaconda/lib/python3.11/site-packages/numpy/lib/function_base.py:4573: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
/data/ephraim/downloads/anaconda/lib/python3.11/site-packages/numpy/lib/function_base.py:4573: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
/data/ephraim/downloads/anaconda/lib/python3.11/site-packages/numpy/lib/function_base.py:4573: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


In [29]:
for noisetype in NOISES:
    noise_mine = mine[mine["noise_type"]==noisetype]
    minestats = noise_mine.describe()[1:3]
    minestats = minestats.assign(alg="ours")
    
    noisedf_storm_5 =  df_storm_5[df_storm_5["noise_type"]==noisetype]
    df_storm_5stats = noisedf_storm_5[cols].describe()[1:3]
    df_storm_5stats = df_storm_5stats.assign(alg="storm")
    
    noise_df_sg_5 = df_sg_5[df_sg_5["noise_type"]==noisetype]
    df_sg_5stats = noise_df_sg_5[cols].describe()[1:3]
    df_sg_5stats = df_sg_5stats.assign(alg="sgmse")
    statsdf = pd.concat([minestats,df_sg_5stats,df_storm_5stats])
    statsdf.to_excel(f"/data/ephraim/datasets/known_noise/sgmse/exp_i/analysis/i_real/stats_{noisetype}.xlsx")

/data/ephraim/downloads/anaconda/lib/python3.11/site-packages/numpy/lib/function_base.py:4573: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
/data/ephraim/downloads/anaconda/lib/python3.11/site-packages/numpy/lib/function_base.py:4573: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
/data/ephraim/downloads/anaconda/lib/python3.11/site-packages/numpy/lib/function_base.py:4573: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
/data/ephraim/downloads/anaconda/lib/python3.11/site-packages/numpy/lib/function_base.py:4573: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
/data/ephraim/downloads/anaconda/lib/python3.11/site-packages/numpy/lib/function_base.py:4573: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
/data/ephraim/downloads/anaconda/lib/python3.11/site-packages/numpy/lib/function_base.py:4573: RuntimeWarning: inva

In [129]:
mine[mine["dir"]=="b"]["filename"].values[0]

'/data/ephraim/datasets/known_noise/undiff/exp_ar_i_095/b/enhanced_60/snr5/s0.1/digits_snr5_power0.0032503484660146675_var0.0003345966415635422.wav'

In [107]:
mine

,dir,name,stoi,input_stoi,pesq,input_pesq,OVRL,SIG,BAK,si_sdr,sdr,sar,sir,noise_type,filename
0,a,s0.3,0.939825,0.924115,1.383912,1.305760,2.659904,3.536586,2.907966,4.052824,4.122058,4.122058,inf,Car,/data/ephraim/datasets/known_noise/undiff/exp_...
1,a,s0.11,0.823307,0.720964,1.280107,1.057805,3.182914,3.478178,4.028701,9.817067,10.241671,10.241671,inf,Babble,/data/ephraim/datasets/known_noise/undiff/exp_...
2,a,s0.12,0.817155,0.758443,1.240639,1.053132,2.381563,3.518393,2.331776,8.819594,9.239682,9.239682,inf,Room,/data/ephraim/datasets/known_noise/undiff/exp_...
3,a,s0.11,0.799380,0.699227,1.310608,1.032454,2.753213,3.355914,3.358786,9.963297,10.560804,10.560804,inf,Factory,/data/ephraim/datasets/known_noise/undiff/exp_...
4,a,s0.1,0.817351,0.719912,1.360432,1.038966,3.067470,3.371378,3.958709,10.687693,11.307800,11.307800,inf,Speech,/data/ephraim/datasets/known_noise/undiff/exp_...
5,b,s0.5,0.966272,0.949485,1.484406,1.450840,2.822509,3.554529,3.287671,3.634245,3.677216,3.677216,inf,Car,/data/ephraim/datasets/known_noise/undiff/exp_...
6,b,s0.1,0.919819,0.811560,1.344813,1.094499,2.490880,2.983444,3.459883,9.103237,10.111362,10.111362,inf,Babble,/data/ephraim/datasets/known_noise/undiff/exp_...
7,b,s0.11,0.927087,0.849798,1.350227,1.098094,2.775957,3.461169,3.243907,8.437196,9.193236,9.193236,inf,Room,/data/ephraim/datasets/known_noise/undiff/exp_...
8,b,s0.2,0.889186,0.788819,1.303768,1.050884,2.396726,3.502589,2.550169,8.345171,8.744496,8.744496,inf,Factory,/data/ephraim/datasets/known_noise/undiff/exp_...
9,b,s0.1,0.921975,0.814749,1.410096,1.070846,3.068494,3.506921,3.792855,9.492676,10.583368,10.583368,inf,Speech,/data/ephraim/datasets/known_noise/undiff/exp_...


In [109]:
mine["filename"][2]

'/data/ephraim/datasets/known_noise/undiff/exp_ar_i_real/a/enhanced_60/snr5/s0.12/noiseRoom_digits_snr5_power0.0030414224602282047_var4.565821647644043.wav'

In [30]:
import shutil
for i in range(len(mine["dir"])):
    wavpath = mine["filename"][i]
    dir_ = mine["dir"][i]
    noisetype = mine["noise_type"][i]
    # print(wavpath)
    name = f"{dir_}_{noisetype}_snr5_ours.wav"
    # print(name)
    dst = f"/data/ephraim/datasets/known_noise/sgmse/exp_i/analysis/i_real/wavs/{name}"
    shutil.copyfile(wavpath, dst)
    # print(dst)

In [31]:
for i in df_sg_5.index.values:
    wavpath = df_sg_5["filename"][i]
    dir_ = df_sg_5["dir"][i]
    noisetype = df_sg_5["noise_type"][i]
    # print(wavpath)
    name = f"{dir_}_{noisetype}_snr5_sgmse.wav"
    # print(name)
    dst = f"/data/ephraim/datasets/known_noise/sgmse/exp_i/analysis/i_real/wavs/{name}"
    shutil.copyfile(wavpath, dst)
    # print(dst)

In [32]:
for i in df_storm_5.index.values:
    wavpath = df_storm_5["filename"][i]
    dir_ = df_storm_5["dir"][i]
    noisetype = df_storm_5["noise_type"][i]
    # print(wavpath)
    name = f"{dir_}_{noisetype}_snr5_storm.wav"
    # print(name)
    dst = f"/data/ephraim/datasets/known_noise/sgmse/exp_i/analysis/i_real/wavs/{name}"
    shutil.copyfile(wavpath, dst)
    # print(dst)

In [33]:
df_noisy= df_noisy.reset_index(drop=True)

In [34]:
df_noisy5 = df_noisy[df_noisy["snr"]=="5"]
for i in df_noisy5.index.values:
    # print(i)
    wavpath = df_noisy5["filename"][i]
    dir_ = df_noisy5["dir"][i]
    noisetype = df_noisy5["noise_type"][i]
    # print(wavpath)
    name = f"{dir_}_{noisetype}_snr5_noisy.wav"
    # print("name:", name)
    dst = f"/data/ephraim/datasets/known_noise/sgmse/exp_i/analysis/i_real/wavs/{name}"
    shutil.copyfile(wavpath, dst)
    # print(dst)

In [35]:
from glob import glob
wavs = glob("/data/ephraim/datasets/known_noise/sgmse/exp_i/clean_i_real/*.wav")
for wavpath in wavs:
    c = Path(wavpath).name.split("_")[0]
    if "snr5" in wavpath:
        noisetype = Path(wavpath).name.split("noise")[1].split("_")[0]
        name = f"{c}_{noisetype}_snr5_clean.wav"
        dst = f"/data/ephraim/datasets/known_noise/sgmse/exp_i/analysis/i_real/wavs/{name}"
        shutil.copyfile(wavpath, dst)
        # print(name)

In [36]:
import torchaudio

from audio_tools2  import *

def calc_vad(f, verbose=False):
    test_file=f
    fs,s = read_wav(test_file)
    win_len = int(fs*0.025)
    hop_len = int(fs*0.010)
    sframes = enframe(s,win_len,hop_len) # rows: frame index, cols: each frame
    if verbose:
        plot_this(compute_log_nrg(sframes))

    # percent_high_nrg is the VAD context ratio. It helps smooth the
    # output VAD decisions. Higher values are more strict.
    percent_high_nrg = 0.5

    vad = nrg_vad(sframes,percent_high_nrg)

    if verbose:
        plot_these(deframe(vad,win_len,hop_len),s)
    return deframe(vad,win_len,hop_len) 

wavs = glob("/data/ephraim/datasets/known_noise/sgmse/exp_i/clean_synth_i95/*.wav")
for wavpath in wavs:
    c = Path(wavpath).name.split("_")[0]
    if "snr5" in wavpath:
        speech, sr = torchaudio.load(wavpath)
        
        vaded_signal = calc_vad(wavpath)[0:speech.shape[1],:]
        vaded_signal_torch = (speech[0][vaded_signal.T[0]>0])
        vaded_signal_torch = torch.unsqueeze(vaded_signal_torch, dim=0)
        # raise Exception
        simple_power =  1 / speech.shape[1] * torch.sum(speech**2)
        clean_power = float( 1 / vaded_signal_torch.shape[1] * torch.sum(vaded_signal_torch**2))
        print(c, simple_power,clean_power)

h tensor(0.0013) 0.0020746218506246805
a tensor(0.0069) 0.009617823176085949
f tensor(0.0005) 0.0007283489685505629
i tensor(0.0027) 0.004131182562559843
g tensor(0.0003) 0.00040919758612290025
j tensor(0.0003) 0.0004039954219479114
c tensor(0.0040) 0.006217417772859335
e tensor(0.0003) 0.0003898124268744141
b tensor(0.0071) 0.010278504341840744
d tensor(0.0026) 0.004266192205250263


In [199]:
group = ["d","e","f","h","j"]
group = ["b"]

minestats = mine[mine["dir"].isin(group)].describe()[1:2]
minestats = minestats.assign(alg="ours")
df_storm_5stats = df_storm_5[df_storm_5["dir"].isin(group)][cols].describe()[1:2]
df_storm_5stats = df_storm_5stats.assign(alg="storm")
df_sg_5stats = df_sg_5[df_sg_5["dir"].isin(group)][cols].describe()[1:2]
df_sg_5stats = df_sg_5stats.assign(alg="sgmse")
statsdf = pd.concat([minestats,df_sg_5stats,df_storm_5stats])
statsdf

/data/ephraim/downloads/anaconda/lib/python3.11/site-packages/numpy/lib/function_base.py:4573: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
/data/ephraim/downloads/anaconda/lib/python3.11/site-packages/numpy/lib/function_base.py:4573: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
/data/ephraim/downloads/anaconda/lib/python3.11/site-packages/numpy/lib/function_base.py:4573: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


,stoi,input_stoi,pesq,input_pesq,OVRL,SIG,BAK,si_sdr,sdr,sar,sir,alg
mean,0.915586,0.852672,1.310969,1.081788,3.001991,3.420187,3.770491,7.701058,8.916986,8.916986,inf,ours
mean,0.956077,0.852672,1.405901,1.081788,3.014956,3.582134,3.592267,9.047626,9.381276,9.381276,inf,sgmse
mean,0.953902,0.852672,1.545146,1.081788,3.269887,3.533363,4.084091,11.197200,12.140747,12.140747,inf,storm


In [194]:
df_storm_5[df_storm_5["dir"].isin(["a","b","c","i"])][cols]

,dir,name,stoi,input_stoi,pesq,input_pesq,OVRL,SIG,BAK,si_sdr,sdr,sar,sir,filename
0,a,digits_snr5_power0.0030414227369186283_var0.00...,0.884283,0.771595,1.397562,1.044373,3.208131,3.465962,4.085089,10.673426,11.061013,11.061013,inf,/data/ephraim/datasets/known_noise/sgmse/exp_i...
1,b,digits_snr5_power0.0032503484660146675_var0.00...,0.953902,0.852672,1.545146,1.081788,3.269887,3.533363,4.084091,11.197200,12.140747,12.140747,inf,/data/ephraim/datasets/known_noise/sgmse/exp_i...
2,c,digits_snr5_power0.001966120132704691_var0.000...,0.904474,0.794392,1.388913,1.065842,3.038718,3.343249,3.911286,11.112709,11.491630,11.491630,inf,/data/ephraim/datasets/known_noise/sgmse/exp_i...
8,i,digits_snr5_power0.001306394632766015_var0.000...,0.856860,0.767430,1.375588,1.119411,3.308637,3.578728,4.094880,9.612803,10.185336,10.185336,inf,/data/ephraim/datasets/known_noise/sgmse/exp_i...
